## Retrieve patient count with Sleep Apnea condition, device exposure & procedure

1) Setup the imports

In [ ]:
 # Path has to be set before importing pyqe
import sys, os 
sys.path.append(os.path.join(sys.path[0],'..', '..'))
 
import json
from pyqe.api.query import Query
from pyqe.api.result import Result
from pyqe.api.concept_query import ConceptSet, ConceptQuery
from pyqe.ql.interaction import Interactions
from pyqe.ql.criteria_group import CriteriaGroup
from pyqe.ql.attribute import Attribute, Constraint, Expression
from pyqe.types.enum_types import ComparisonOperator, MatchCriteria, Domain

2) Define query with name

In [ ]:
query = Query('Patient_With_Sleep_Apnea_Count')
await query.set_study("703c5d8a-a1d9-4d42-a314-5b9aad513390")

3) Generate standard concept ids from concept codes

In [ ]:
conceptQuery = ConceptQuery()
sleep_apnea_dx_concept_codes = ['313459', '41975002', '79280005', '78275009']
sleep_apnea_dx_concept_ids = [(await conceptQuery.get_standard_concept_ids(concept_code))[0]['conceptId'] for concept_code in sleep_apnea_dx_concept_codes]

sleep_apnea_device_concept_codes = ['A7034', 'A9270', 'E0470', 'E0471', 'E0601']
sleep_apnea_device_concept_ids = [(await conceptQuery.get_standard_concept_ids(concept_code))[0]['conceptId'] for concept_code in sleep_apnea_device_concept_codes]

sleep_apnea_cpt_procedure_concept_code = '103750000'
sleep_apnea_cpt_procedure_concept_id = (await conceptQuery.get_standard_concept_ids(sleep_apnea_cpt_procedure_concept_code))[0]['conceptId']

4) Define sleep apnea concept set filters

In [ ]:
sleep_apnea_dx = ConceptSet('Sleep Apnea Dx', Domain.CONDITION, ['73430006', '41975002', '79280005', '78275009'])
sleep_apnea_dx_condition = Interactions.ConditionOccurrence(sleep_apnea_dx.name)
sleep_apnea_dx_condition.add_concept_set(sleep_apnea_dx)

sleep_apnea_device = ConceptSet('Sleep Apnea Device', Domain.DEVICE, ['A7034', 'A9270', 'E0470', 'E0471', 'E0601'])
sleep_apnea_device_exposure = Interactions.DeviceExposure(sleep_apnea_device.name)
sleep_apnea_device_exposure.add_concept_set(sleep_apnea_device)

sleep_apnea_cpt = ConceptSet('Sleep Apnea CPT4', Domain.PROCEDURE, ['95811'])
sleep_apnea_cpt_procedure = Interactions.ProcedureOccurrence(sleep_apnea_cpt.name)
sleep_apnea_cpt_procedure.add_concept_set(sleep_apnea_cpt)

5) Define patient count filter request by adding criteria group containing the filters

In [ ]:
# Create criteria group to create query matching all filters
query.add_criteria_group(CriteriaGroup(MatchCriteria.ALL, [sleep_apnea_dx_condition, sleep_apnea_device_exposure, sleep_apnea_cpt_procedure]))
request = query.get_patient_count_filter()
# Print request
# print(f'\nRequest: {json.dumps(request)}')

6) Get patient count with request

In [ ]:
patient_count = await Result().get_patient_count(request)
print(
    f'\nTotal patients with Sleep Apnea condition, device & CPT4 procedure: {patient_count}')